In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/Abisheka06/Solar_Panel_Dataset.git
%cd Solar_Panel_Dataset

Cloning into 'Solar_Panel_Dataset'...
remote: Enumerating objects: 809, done.
remote: Total 809 (delta 0), reused 0 (delta 0), pack-reused 809 (from 1)
Receiving objects: 100% (809/809), 269.00 MiB | 36.29 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (891/891), done.
/content/Solar_Panel_Dataset


In [ ]:
import os
import shutil

source_dir = '/content/Solar_Panel_Dataset'
clean_dir = '/content/Clean_Solar_Panel_Dataset'

os.makedirs(clean_dir, exist_ok=True)

# Copy only folders that are actual class names (not starting with '.')
for item in os.listdir(source_dir):
    s = os.path.join(source_dir, item)
    d = os.path.join(clean_dir, item)
    if os.path.isdir(s) and not item.startswith('.'):
        shutil.copytree(s, d, dirs_exist_ok=True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

# Set paths
data_dir = "/content/Clean_Solar_Panel_Dataset"

# Image parameters
img_size = (224, 224)
batch_size = 32

# Data generators with split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
print(train_gen.class_indices)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Number of classes
num_classes = len(train_gen.class_indices)
print(num_classes)

# Build model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_gen, validation_data=val_gen, epochs=30)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"\nValidation Accuracy: {acc*100:.2f}%")

# Save the model
model.save("solar_panel_classifier.keras")
print("\nModel Saved")


Found 711 images belonging to 6 classes.
{'Bird-drop': 0, 'Clean': 1, 'Dusty': 2, 'Electrical-damage': 3, 'Physical-Damage': 4, 'Snow-Covered': 5}
Found 174 images belonging to 6 classes.
6
Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.1240 - loss: 2.7515 - val_accuracy: 0.2471 - val_loss: 1.6957
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.2622 - loss: 1.9861 - val_accuracy: 0.3448 - val_loss: 1.5211
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.3224 - loss: 1.8155 - val_accuracy: 0.4023 - val_loss: 1.4451
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.3510 - loss: 1.6918 - val_accuracy: 0.4483 - val_loss: 1.3750
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.3648 - loss: 1.6287 - val_accuracy: 0.5172 - val_loss: 1.2692
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.4069 - loss: 1.5532 - val_accuracy: 0.5345 - val_loss: 1.2309
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - ac

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("solar_panel_classifier.keras")